In [1]:
!pip install -qq transformers[sentencepiece]
!pip install -qq datasets

     |████████████████████████████████| 2.5MB 16.8MB/s 
     |████████████████████████████████| 3.3MB 43.7MB/s 
     |████████████████████████████████| 901kB 39.4MB/s 
     |████████████████████████████████| 1.1MB 43.9MB/s 
     |████████████████████████████████| 266kB 27.8MB/s 
     |████████████████████████████████| 245kB 43.0MB/s 
     |████████████████████████████████| 122kB 48.1MB/s 


## Bert Tokenizer

In [2]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
print(type(tokenizer))
print(tokenizer)


<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


## Auto Models from HuggingFace

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
print(type(tokenizer))
print(tokenizer)


<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


## Try out the tokenizer

In [4]:
inputs = tokenizer('hi my name is Mike from Udemy')

print(type(inputs), type(inputs.data))
print(inputs.data)

<class 'transformers.tokenization_utils_base.BatchEncoding'> <class 'dict'>
{'input_ids': [101, 20844, 1139, 1271, 1110, 2639, 1121, 158, 2007, 4527, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [5]:
for idx, tok in enumerate(tokenizer.vocab.keys()):
    print(f'{tok}: {tokenizer.vocab[tok]}')
    if(idx > 5): break

China: 1975
alpha: 11164
Mobile: 8410
roses: 16760
##ntal: 21739
ɾ: 364
Rwanda: 18845


In [6]:
print(tokenizer.decode(inputs.input_ids))
print(tokenizer.convert_ids_to_tokens(inputs.input_ids))

[CLS] hi my name is Mike from Udemy [SEP]
['[CLS]', 'hi', 'my', 'name', 'is', 'Mike', 'from', 'U', '##de', '##my', '[SEP]']


## Sentiment Classification
Using the Pipeline from Hugginface
(Of course we will have to finetune the model on our data)

In [7]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis')
classifier('I want to attend a huggingFace class to learn more on using transformer models')

[{'label': 'NEGATIVE', 'score': 0.9842770099639893}]

In [ ]:
classifier(['I want to attend a huggingFace class',
           'I want to learn more on using transformer models'])

[{'label': 'POSITIVE', 'score': 0.8202962279319763},
 {'label': 'NEGATIVE', 'score': 0.9910460710525513}]

In [8]:
classifier.model.name_or_path, classifier.modelcard

('distilbert-base-uncased-finetuned-sst-2-english', None)

**Let's try another BERT-Model:**

In [9]:
classifier = pipeline("sentiment-analysis", model='oliverguhr/german-sentiment-bert')
print(f'Results: {classifier("Das klingt doch sehr gut!")}')
print(classifier.model.name_or_path, classifier.modelcard)


Results: [{'label': 'positive', 'score': 0.9613783359527588}]
oliverguhr/german-sentiment-bert None


## Zero Shot Classification

In [10]:
classifier = pipeline("zero-shot-classification")
classifier("This course is about the Transformers Model Library",
           candidate_labels=['education', 'politics', 'business'],
          )

{'labels': ['education', 'business', 'politics'],
 'scores': [0.7934281826019287, 0.15321235358715057, 0.05335952714085579],
 'sequence': 'This course is about the Transformers Model Library'}

In [11]:
print(classifier.model.name_or_path, classifier.modelcard)

facebook/bart-large-mnli None


## Textgeneration

In [12]:
generator = pipeline("text-generation")
generator("In this course, we will teach you how to....")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to....\n\nCreate Your Own Web Application\n\nThis is a two-phase course. In early November of 2011, for only $0.99, I had successfully created a website using JavaScript which'}]

In [13]:
generator = pipeline("text-generation", model='distilgpt2')
generator("In this course, we will teach you how to....",
         max_length=30,
         num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to....'},
 {'generated_text': 'In this course, we will teach you how to....\n\nIf you look at how you can learn to be productive in your day job, it'}]

https://huggingface.co/blog/how-to-generate

## Language Modelling
"Fill in the blanks"

In [14]:
unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

[{'score': 0.19619838893413544,
  'sequence': 'This course will teach you all about mathematical models.',
  'token': 30412,
  'token_str': ' mathematical'},
 {'score': 0.040527306497097015,
  'sequence': 'This course will teach you all about computational models.',
  'token': 38163,
  'token_str': ' computational'}]

## Token Classification
What does `grouped_entities` do?
Example: NER - named entity recognition

In [15]:
ner = pipeline("ner", grouped_entities=True)
ner("My name is Bert and I work at Hugging Face in Brooklyn.")

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:155: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  f'`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="{aggregation_strategy}"` instead.'


[{'end': 15,
  'entity_group': 'PER',
  'score': 0.99858105,
  'start': 11,
  'word': 'Bert'},
 {'end': 42,
  'entity_group': 'ORG',
  'score': 0.97348887,
  'start': 30,
  'word': 'Hugging Face'},
 {'end': 54,
  'entity_group': 'LOC',
  'score': 0.99345374,
  'start': 46,
  'word': 'Brooklyn'}]

In [16]:
ner = pipeline("ner", grouped_entities=False)
ner("My name is Bert and I work at Hugging Face in Brooklyn.")

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:155: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.NONE"` instead.
  f'`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="{aggregation_strategy}"` instead.'


[{'end': 15,
  'entity': 'I-PER',
  'index': 4,
  'score': 0.99858105,
  'start': 11,
  'word': 'Bert'},
 {'end': 32,
  'entity': 'I-ORG',
  'index': 9,
  'score': 0.9760045,
  'start': 30,
  'word': 'Hu'},
 {'end': 37,
  'entity': 'I-ORG',
  'index': 10,
  'score': 0.96469575,
  'start': 32,
  'word': '##gging'},
 {'end': 42,
  'entity': 'I-ORG',
  'index': 11,
  'score': 0.97976637,
  'start': 38,
  'word': 'Face'},
 {'end': 54,
  'entity': 'I-LOC',
  'index': 13,
  'score': 0.99345374,
  'start': 46,
  'word': 'Brooklyn'}]

## Question Answering

In [17]:
question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I Work?",
    context="My name is Bert and I work at HugginFace in Memphis")

{'answer': 'HugginFace', 'end': 40, 'score': 0.686631441116333, 'start': 30}

## Text Summarization

In [18]:
summarizer = pipeline("summarization")
summarizer("""
1.1	Die VERKAUFENDE PARTEI ist aufgrund des Kaufvertrags vom 13.11.1990, der Urkunde vom 4.2.1994 sowie der Urkunde vom 3.11.1994 (verbüchert zu TZ 7452/1994) Eigentümerin zu 154/10810-Anteilen (B-LNR 14) an der Liegenschaft EZ 3439, Katastralgemeinde 51215 Lichtenegg, Bezirksgericht Wels mit den ihr inneliegenden Grundstücken 840/26 im Ausmaß von 3800 m² und 840/32 im Ausmaß von von 173 m² (im Folgenden kurz: die „LIEGENSCHAFT“). 
1.2	Gegenstand dieses Kaufvertrags (im Folgenden kurz: der „KAUFGEGENSTAND“) sind die der VERKAUFENDEN PARTEI gehörigen 154/10810-Anteile an der LIEGENSCHAFT, mit welchen untrennbar Wohnungseigentum an Einheit 11 verbunden ist (B-LNR 14). Die kaufgegenständliche Wohnung weist die im Maklerexposé (Beilage ./1.2) angeführten Eigenschaften auf. 

""")

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


[{'summary_text': ' Die VERKAUFENDE PARTEI ist aufgrund des Kaufvertrags vom 13.11.1990, der Urkunde vom 4.2.1994 sowie . The Verkaufenden PartyeI is gegenen 154/10810-Anteile an der LIEGENSCHAFT . Die kaufgegenständliche Wohnung weist die im Maklerexposé (Beilage./1.2) angeführten Eigenschaften auf.2).'}]

## Translation

In [19]:
translator = pipeline("translation", model='Helsinki-NLP/opus-mt-fr-en')
translator("Ce cours est produit par Hugging Face")

[{'translation_text': 'This course is produced by Hugging Face'}]